# Small System, ~30 atoms

ANI2x simulating a small peptide in vacuum

In [ ]:
# Import libraries
from openmm.app import *
from openmm import *
from openmm.unit import *
import openmmml
import sys

# Load small alanine tripeptide system
pdb = PDBFile("../pdbs/aaa_capped.pdb")
potential = openmmml.MLPotential('ani2x')
system = potential.createSystem(
    pdb.topology,
    nonbondedMethod=PME,
    nonbondedCutoff=1*nanometer,
    constraints=HBonds,
    hydrogenMass=1*amu # required
)

# Setup simulation
integrator = LangevinIntegrator(
    300 * kelvin, 
    1 / picosecond, 
    2.0 * femtosecond,
)
simulation = Simulation(
    pdb.topology,
    system,
    integrator,
)
STEPS = 1000
simulation.reporters.append(
    StateDataReporter(
        sys.stdout, 
        30, 
        totalSteps=STEPS,
        speed=True,
        progress=True,
        remainingTime=True,
    )
)
simulation.context.setPositions(pdb.positions)

# Minimise and run
simulation.minimizeEnergy()
simulation.step(STEPS)
print("done")

# Large Solvated System, Mixed ANI/AMBER, ~2500 atoms

ANI2x is used for the peptide, and AMBER for the solvent.

ANI2x cannot be used alone for such a large system, as it uses too much memory. I read somewhere that memory usage is ~On^2 for number of atoms

In [ ]:
# Import libraries
from openmm.app import *
from openmm import *
from openmm.unit import *
import openmmml
import sys

# Load solvated, equilibrated system
pdb = PDBFile("../pdbs_equilibrated/aaa/aaa_capped_amber_equilibrated.pdb")

# Select peptide (non-water) atoms to be simulated by ANI
peptide_atoms = [atom.index for atom in pdb.topology.atoms() if atom.residue.name != "HOH"]

# Create amber system
amber_system = ForceField(
    'amber14-all.xml',
    'amber14/tip3p.xml'
).createSystem(
    pdb.topology,
    nonbondedMethod=PME,
    nonbondedCutoff=1*nanometer,
    constraints=HBonds,
    hydrogenMass=1*amu # required
)

# Setup mixed system
potential = openmmml.MLPotential('ani2x')
system = potential.createMixedSystem(
    pdb.topology,
    amber_system,
    peptide_atoms
)

# Setup simulation
integrator = LangevinIntegrator(
    300 * kelvin, 
    1 / picosecond, 
    2 * femtosecond, # higher stepsizes are unstable
)
simulation = Simulation(
    pdb.topology,
    system,
    integrator,
)
STEPS = 1000
simulation.reporters.append(
    StateDataReporter(
        sys.stdout, 
        30, 
        totalSteps=STEPS,
        speed=True,
        progress=True,
        remainingTime=True,
    )
)
simulation.context.setPositions(pdb.positions)

# Minimise and run
simulation.minimizeEnergy()
simulation.step(STEPS)
print("done")
